In [19]:
print("Hello, World!")
from openai import OpenAI
import os
from dotenv import load_dotenv
from pprint import pprint
import json
# # After vars(call), it becomes a dictionary:
# tool_calls_dict = [vars(call) for call in tool_calls]
# tool_name = tool_calls_dict[0]["function"].name
# args = json.loads(tool_calls_dict[0]["function"].arguments)
# Pretty print the tool calls

Hello, World!


In [ ]:
# define all the tools in function specification of the agent
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["c", "f"]},
                },
                "required": ["location", "unit"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "execute_sql_query",
            "description": "Execute a SQL query on a database",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The SQL query to execute"
                    },
                    "database": {
                        "type": "string",
                        "enum": ["users", "products", "orders"],
                        "description": "The database to query"
                    }
                },
                "required": ["query", "database"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_react_component",
            "description": "Generate a React component based on specifications",
            "parameters": {
                "type": "object",
                "properties": {
                    "component_name": {
                        "type": "string",
                        "description": "Name of the React component"
                    },
                    "features": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of features/requirements for the component"
                    },
                    "styling": {
                        "type": "string",
                        "enum": ["css", "tailwind", "styled-components"],
                        "description": "Preferred styling method"
                    }
                },
                "required": ["component_name", "features"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time for a specific timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "Timezone (e.g., 'UTC', 'America/New_York')"
                    },
                    "format": {
                        "type": "string",
                        "enum": ["12h", "24h"],
                        "default": "24h"
                    }
                },
                "required": ["timezone"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_news",
            "description": "Get latest news articles based on topic and source",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "News topic or search term"
                    },
                    "source": {
                        "type": "string",
                        "enum": ["tech", "business", "sports", "general"],
                        "description": "News category"
                    },
                    "limit": {
                        "type": "integer",
                        "minimum": 1,
                        "maximum": 10,
                        "default": 5,
                        "description": "Number of articles to return"
                    }
                },
                "required": ["topic"]
            }
        }
    }
]


def get_weather(location, unit):
    print(f"The weather in {location} with {unit} is sunny and 75 degrees.")
    return {"weather": "sunny", "temperature": 31}

def execute_sql_query(query, database):
    # Implementation for SQL query execution
    return {"results": f"Query executed on {database}: {query}"}

def generate_react_component(component_name, features, styling="css"):
    # Implementation for React component generation
    return {
        "code": f"Generated React component {component_name} with {styling}",
        "features": features
    }

def get_current_time(timezone, format="24h"):
    from datetime import datetime
    import pytz
    # Implementation for time retrieval
    tz = pytz.timezone(timezone)
    current_time = datetime.now(tz)
    return {
        "time": current_time.strftime("%H:%M" if format == "24h" else "%I:%M %p"),
        "timezone": timezone
    }

def get_news(topic, source="general", limit=5):
    # Implementation for news retrieval
    return {
        "articles": f"Retrieved {limit} articles about {topic} from {source}"
    }

In [18]:
# this is a tool calling example
load_dotenv()
the_key = os.getenv("OPENAI_API_KEY")
# print(the_key)

client = OpenAI(api_key=the_key,
                #base_url="https://api.openai.com/"
                )

tools_str = json.dumps(tools)
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"""You are a helpful assistant that can answer questions and perform tasks. You have access to the following tools: 
         {tools_str}
         """},
        {"role": "user", "content": "What's the weather like in Paris today? pls make a react app with weather component?"}
        ],
    tools=tools,
)

print(completion.choices[0].message)
print("--------------------------------")
assistant_message = completion.choices[0].message
tool_calls = completion.choices[0].message.tool_calls
print("assistant_message" , assistant_message)
print("tool_calls" , tool_calls)
print("--------------------------------")
if tool_calls:
    for tool_call in tool_calls:
        args = json.loads(tool_call.function.arguments)
        tool_name = tool_call.function.name
        if tool_name == "get_weather":
            print("get_weather")
            print(args)
            result = get_weather(
                location=args["location"],
                unit=args["unit"]
            )
        elif tool_name == "execute_sql_query":
            print("execute_sql_query")
            print(args)
            result = execute_sql_query(
                query=args["query"],
                database=args["database"]
            )
        elif tool_name == "generate_react_component":
            print("generate_react_component")
            print(args)
            result = generate_react_component(
                component_name=args["component_name"],
                features=args["features"],
                styling=args["styling"]
            )  
        elif tool_name == "get_current_time":
            print("get_current_time")
            print(args)
            result = get_current_time(
                timezone=args["timezone"],
                format=args["format"]
            )
        elif tool_name == "get_news":
            print("get_news")
            print(args)
            result = get_news(
                topic=args["topic"],
                source=args["source"],
                limit=args["limit"]
            )
        print(result)


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pZNdXoX24zHL5aLtoAffIU4f', function=Function(arguments='{"location":"Paris","unit":"c"}', name='get_weather'), type='function')])
--------------------------------
assistant_message ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pZNdXoX24zHL5aLtoAffIU4f', function=Function(arguments='{"location":"Paris","unit":"c"}', name='get_weather'), type='function')])
tool_calls [ChatCompletionMessageToolCall(id='call_pZNdXoX24zHL5aLtoAffIU4f', function=Function(arguments='{"location":"Paris","unit":"c"}', name='get_weather'), type='function')]
--------------------------------
get_weather
{'location': 'Paris', 'unit': 'c'}
The weather in Paris with c is sunny and 75 degrees.
{'weather': 'sunny', 'temperature': 31}
